# Rice yield prediction project

This is a part of field based experiment conducted from 2018-2022 at International Rice Research Institute, Philipinnes where we want to compare how well multi-spectral data taken from drone remote sensing platform can help predict the complex trait(yield) compared to classical model i.e GBLUP(Genomic best linear unbiased predictor) which uses genomic data and this research is a chapter for master's thesis work.<br>
 In plant breeding the accuracy i.e correlation between the predicted value of germplasm and its actual value(response"yield")  is very important to assess the top performing germplasm. Based on estimated breeding value of germplasm they are selected in a breeding program which later is released as a variety commercially to grower and farmers. <br>
Unlike other field, accuracy tend to be controlled by several factor in plant breeding because the accuracy of model to predict the trait is affected highly by the heritability of trait and since the complex trait such as yield, biomass has very low heritability, even the best model tend to have low accuracy and it cannot be more than the value of heritability of the trait. (Heritability is the proportion of phenotypic variance among individuals in a population that is due to heritable genetic effects which gets transferred to the progeny)

In [2]:
import numpy as np
import pandas as pd
import os

C:\Users\Ashmita\AppData\Local\Temp\ipykernel_12644\2443873611.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# loading 2022 phenotypic data
phenoComb_2022_5b= pd.read_csv('phenoComb_2022_5b.csv')

#Remove 'GERMPLASMNAME' and PLOTNO column from dataframe 
phenoComb_2022_5b = phenoComb_2022_5b.drop(['GERMPLASMNAME', 'PLOTNO'], axis=1)


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_val_predict

#splitting train and test and using 10 fold cv further to validate model
X = phenoComb_2022_5b.drop(columns=['YLDTON'], axis=1) #yield measured in ton is response
y = phenoComb_2022_5b['YLDTON']  
imputer = SimpleImputer(strategy='mean') # imputing NA in response
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1)).ravel()
# Initialize the RandomForestRegressor
rf_regressor = RandomForestRegressor()
scores = cross_val_score(rf_regressor, X, y, cv=10, scoring='neg_mean_squared_error')
print("MSE scores for each fold:", -scores)
print("Average MSE:", -scores.mean())
#obtaining accuracy here
y_pred = cross_val_predict(rf_regressor, X, y, cv=10)
correlation = np.corrcoef(y, y_pred)[0, 1]
print("Correlation coefficient:", correlation)

MSE scores for each fold: [1.07650417 1.21235855 0.7709177  1.07681812 1.07364322 1.1198846
 1.15785374 0.85336752 1.48149431 1.50269497]
Average MSE: 1.1325536883214282
Correlation coefficient: 0.427143679765113


In [22]:
from sklearn.model_selection import GridSearchCV

# Defining the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],      
    'max_depth': [10, 20, 30],      
    'min_samples_split': [2, 5, 10],     
    'min_samples_leaf': [1, 2, 4]      
}

# Initialize the RandomForestRegressor
rf_regressor = RandomForestRegressor()

# Initialize the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Fitting the grid search to the 2022_5b cohort data
grid_search.fit(X, y_imputed)

# Print the best parameters and the score
print("Best parameters:", grid_search.best_params_)
print("Best score (negative MSE):", grid_search.best_score_)


Fitting 10 folds for each of 81 candidates, totalling 810 fits
Best parameters: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Best score (negative MSE): -1.1171116551599414


In [34]:
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Using the best parameters that were found in chunk above  to create a new RandomForestRegressor
best_rf = RandomForestRegressor(
    n_estimators=grid_search.best_params_['n_estimators'],
    max_depth=grid_search.best_params_['max_depth'],
    min_samples_split=grid_search.best_params_['min_samples_split'],
    min_samples_leaf=grid_search.best_params_['min_samples_leaf'],
    random_state=42
)

# Fit the model on the entire training dataset
best_rf.fit(X, y_imputed)

# Evaluateingthe model on the test data that was splitted
from sklearn.metrics import mean_squared_error

y_pred = best_rf.predict(X_test) 
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)
correlation, _ = pearsonr(y_test, y_pred)
print("Pearson Correlation Coefficient:", correlation)

Test MSE: 0.13955917323931682
Pearson Correlation Coefficient: 0.9751337904367676


# performing feature importance just to check which feature are important and this feature importance from scikit uses the MDI method 

In [ ]:
# Feature importance
importances = best_rf.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X.columns.tolist()  
print("Feature ranking:")
for f in range(X.shape[1]):
    print(f"{f + 1}. feature {feature_names[indices[f]]} ({importances[indices[f]]})")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
importances = best_rf.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X.columns.tolist() 
# Selecting the top 60 features
top_n = 60
top_indices = indices[:top_n]
# Plot the feature importances of the top 60 features
plt.figure(figsize=(10, 8)) 
plt.title("Top 60 Feature Importances")
bars = plt.bar(range(top_n), importances[top_indices], color='skyblue')
plt.xticks(range(top_n), [feature_names[i] for i in top_indices], rotation=90)
plt.xlim([-1, top_n])
plt.xlabel('Features')
plt.ylabel('Importance')
plt.tight_layout() 
for bar, imp in zip(bars, importances[top_indices]):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, round(imp, 4), ha='center', va='bottom', fontsize=8, rotation=90, color='black')

plt.show()


# Gradient Boosting decision tree method on 2022 yield prediction data to compare the performance of random forest and gradient boosting decision tree

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_val_predict

#splitting train and test
X = phenoComb_2022_5b.drop(columns=['YLDTON'], axis=1) 
y = phenoComb_2022_5b['YLDTON']  
imputer = SimpleImputer(strategy='mean')
y_imputed = imputer.fit_transform(y.values.reshape(-1, 1)).ravel()

In [5]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Defining the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],      
    'learning_rate': [0.05, 0.1, 0.2],  
    'max_depth': [3, 4, 5],              
    'min_samples_split': [2, 5, 10],       
    'min_samples_leaf': [1, 2, 4]
}

# Initializing the GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor()

# Initializing the GridSearchCV object
grid_search = GridSearchCV(estimator=gb_regressor, param_grid=param_grid, cv=10, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X, y)

# Print the best parameters and the score
print("Best parameters:", grid_search.best_params_)
print("Best negative MSE:", grid_search.best_score_)


Fitting 10 folds for each of 243 candidates, totalling 2430 fits
Best parameters: {'learning_rate': 0.05, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best negative MSE: -1.1465556741095224


In [6]:
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using the best parameters to create a new gradientboostregressor using best parameter from above chunk of code
best_gbd = GradientBoostingRegressor(
    n_estimators=grid_search.best_params_['n_estimators'],
    max_depth=grid_search.best_params_['max_depth'],
    min_samples_split=grid_search.best_params_['min_samples_split'],
    min_samples_leaf=grid_search.best_params_['min_samples_leaf'],
    random_state=42
)

# Fit the model on the entire training dataset
best_gbd.fit(X, y_imputed)

# Evaluate the model on the test dataset
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)

# Calculate the Pearson correlation coefficient
correlation, _ = pearsonr(y_test, y_pred)
print("Pearson Correlation Coefficient:", correlation)

Test MSE: 0.011186986359468411
Pearson Correlation Coefficient: 0.996651208270527


Two sets of model were fiting with 10 fold cv to assess the model performance and further metrices such as MSE and accuracy(relevant to plant breeding) was used to make the conclusion. Best parameters were serched by using gridsearch method which helped to fine tune the model further in both cases.<br>
The MSE from random forest model and gradient boosting decision tree model is 0.13 and 0.011 respectively. The accuracy of random forest model is 0.97 & the GBDT is 0.99. So, the performance of gradient boosting deciison tree is better than random forest in one year of data using only spectral data.<br>
However, the conclusion can only be made after fitting model for all year 2019-2022 and using genomic information in the model in full scale.